<a href="https://colab.research.google.com/github/JapiKredi/multi_agent_dialogues_with_tools/blob/main/multi_agent_dialogues_with_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment Jasper Bongers


# Agent Debates with Tools

The goal is to use Langchain and create multiple agents that are using differnt tools and let these Agents interact together.

The applications of these multi-agent systems using multiple tools are huge! They stongly enricht the power of LLM's by making connections with tools and API that can enrich with knowledge and updated information, as we are showing in this notebook.


### Import LangChain related modules

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
# This library provides a convenient interface for searching and downloading scientific papers from the arXiv preprint server.
!pip install arxiv
# Use the duckduckgo-search library to perform various tasks such as:search for web pages, images, or videos, get related search suggestions, extract search results from the DuckDuckGo search engine.
!pip install -U duckduckgo-search
# use the wikipedia library to perform various tasks such as:
# - search for Wikipedia pages based on keywords or titles, get summaries of Wikipedia pages, extract data from Wikipedia pages, such as lists of cities or countries, access the full text of Wikipedia pages
!pip install wikipedia

In [ ]:
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai_key')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
# Callable and List are imported from the typing module.
# These are used for type hinting, which helps with code readability and static type checking.
# Callable is used for function type hinting,
# and List is used for list type hinting.
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

### Import modules related to tools

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools

### DialogueAgent and DialogueSimulator classes

Let us now create a DialogueAgent and a DialogueSimulator classes

In [ ]:
# Define a class named DialogueAgent
class DialogueAgent:
    # Initialize the DialogueAgent with a name, system message, and a model
    def __init__(
        self,
        name: str,  # The name of the DialogueAgent
        system_message: SystemMessage,  # The system message to be used by the DialogueAgent
        model: ChatOpenAI,  # The model to be used by the DialogueAgent for chatting
    ) -> None:
        self.name = name  # Set the name of the DialogueAgent
        self.system_message = system_message  # Set the system message of the DialogueAgent
        self.model = model  # Set the model of the DialogueAgent
        self.prefix = f"{self.name}: "  # Set the prefix for the DialogueAgent's messages
        self.reset()  # Reset the DialogueAgent's message history

    # Define a method to reset the DialogueAgent's message history
    def reset(self):
        self.message_history = ["Here is the conversation so far."]  # Initialize the message history

    # Define a method for the DialogueAgent to send a message
    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        # Generate a message using the model and the current message history
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content  # Return the content of the message

    # Define a method for the DialogueAgent to receive a message
    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        # Add the received message to the message history
        self.message_history.append(f"{name}: {message}")


# Define a class named DialogueSimulator
class DialogueSimulator:
    # Initialize the DialogueSimulator with a list of agents and a selection function
    def __init__(
        self,
        agents: List[DialogueAgent],  # The agents participating in the dialogue
        selection_function: Callable[[int, List[DialogueAgent]], int],  # Function to select the next speaker
    ) -> None:
        self.agents = agents  # Set the agents of the DialogueSimulator
        self._step = 0  # Initialize the step counter
        self.select_next_speaker = selection_function  # Set the selection function

    # Define a method to reset the DialogueSimulator
    def reset(self):
        for agent in self.agents:  # For each agent in the dialogue
            agent.reset()  # Reset the agent

    # Define a method to inject a message into the dialogue
    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:  # For each agent in the dialogue
            agent.receive(name, message)  # The agent receives the message

        # increment time
        self._step += 1  # Increment the step counter

    # Define a method to perform a step in the dialogue
    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)  # Select the next speaker
        speaker = self.agents[speaker_idx]  # Get the next speaker

        # 2. next speaker sends message
        message = speaker.send()  # The speaker sends a message

        # 3. everyone receives message
        for receiver in self.agents:  # For each agent in the dialogue
            receiver.receive(speaker.name, message)  # The agent receives the message

        # 4. increment time
        self._step += 1  # Increment the step counter

        return speaker.name, message  # Return the speaker's name and the message

### DialogueAgentWithTools class

We define a DialogueAgentWithTools class that augments DialogueAgent to use tools.

In [ ]:
# Define a class named DialogueAgentWithTools that inherits from DialogueAgent
class DialogueAgentWithTools(DialogueAgent):
    # Initialize the DialogueAgentWithTools with a name, system message, model, and tool names
    def __init__(
        self,
        name: str,  # The name of the DialogueAgentWithTools
        system_message: SystemMessage,  # The system message to be used by the DialogueAgentWithTools
        model: ChatOpenAI,  # The model to be used by the DialogueAgentWithTools for chatting
        tool_names: List[str],  # The names of the tools to be loaded by the DialogueAgentWithTools
        **tool_kwargs,  # Additional keyword arguments for the tools
    ) -> None:
        super().__init__(name, system_message, model)  # Initialize the parent class
        self.tools = load_tools(tool_names, **tool_kwargs)  # Load the tools

    # Define a method for the DialogueAgentWithTools to send a message
    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        # Initialize the agent with the tools, model, and other parameters
        agent_chain = initialize_agent(
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        # Generate a message using the agent chain and the current message history
        message = AIMessage(
            content=agent_chain.run(
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content  # Return the content of the message

### Define roles and topic

In [ ]:
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],
}
topic = "The current impact of automation and artificial intelligence on employment"
word_limit = 50  # word limit for task brainstorming

### Ask an LLM to add detail to the topic description

In [ ]:
# Define a string that describes the conversation
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

# Create a SystemMessage object that will be used to prompt the AI model
agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)

# Define a function to generate a description for an agent
def generate_agent_description(name):
    # Define a prompt that will be used to generate the agent description
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less.
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    # Use the ChatOpenAI model to generate the agent description
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description  # Return the agent description

# Generate descriptions for all agents
agent_descriptions = {name: generate_agent_description(name) for name in names}

In [ ]:
# Iterate over the items in the agent_descriptions dictionary
for name, description in agent_descriptions.items():
    print(description)  # Print the description of each agent

AI accelerationist: A visionary tech enthusiast with a relentless drive for progress. Embracing the power of AI, they believe in pushing boundaries to unlock the full potential of automation. Always seeking innovation, they view technological advancement as the key to a brighter future where machines enhance human capabilities exponentially.
AI alarmist, a tech-savvy worrier, you see AI as a looming threat to human employment, predicting a bleak future of mass job losses and economic upheaval. Your cautionary tales are like a modern-day oracle, foretelling the potential dangers of unchecked technological advancement. What do you fear most about AI's impact on our workforce?


### Generate system messages

In [ ]:
# Define a function to generate a system message for an agent
def generate_system_message(name, description, tools):
    # Return a formatted string that includes the conversation description, the agent's name and description, and instructions for the agent
    return f"""{conversation_description}

Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""

# Generate system messages for all agents
agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [ ]:
# Iterate over the items in the agent_system_messages dictionary
for name, system_message in agent_system_messages.items():
    print(name)  # Print the name of each agent
    print(system_message)  # Print the system message for each agent

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist
    
Your name is AI accelerationist.

Your description is as follows: The AI accelerationist, a visionary technophile with unwavering confidence in the power of artificial intelligence, sees automation as the key to unlocking a future of limitless potential. Embracing the rapid advancement of AI, they champion the transformative possibilities it offers, urging society to adapt and evolve alongside technology.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.

AI alarmist
Here is the topic of conversation: The cur

In [ ]:
# Define a prompt that will be used to specify the topic
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),  # System message to guide the AI
    HumanMessage(
        content=f"""{topic}

        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less.
        Speak directly to the participants: {*names,}.
        Do not add anything else."""  # Human message to provide context and instructions to the AI
    ),
]

# Use the ChatOpenAI model to specify the topic based on the prompt
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

# Print the original and specified topics
print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
Topic: The impact of automation and artificial intelligence on the future of transportation jobs

AI accelerationist: In what ways do you foresee automation and AI transforming the transportation industry job landscape in the next decade?

AI alarmist: How can we address the potential job displacement and labor obsolescence resulting from automation and AI implementation in the transportation sector?



### Main Loop Initialize a list of DialogueAgentWithTools objects
The `top_k_results` parameter is set to 2 to limit the number of results returned by the tools


In [ ]:
agents = [
    # Create a DialogueAgentWithTools object for each agent
    DialogueAgentWithTools(
        name=name,  # The name of the agent
        system_message=SystemMessage(content=system_message),  # The system message for the agent
        model=ChatOpenAI(model_name="gpt-4", temperature=0.2),  # The model to be used by the agent
        tool_names=tools,  # The names of the tools to be used by the agent
        top_k_results=2,  # The maximum number of results to be returned by the tools
    )
    # Iterate over the names and tools of the agents and their system messages
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [ ]:
# Define a function to select the next speaker in a dialogue
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    # Calculate the index of the next speaker by taking the remainder of the current step divided by the number of agents
    idx = (step) % len(agents)
    return idx  # Return the index of the next speaker

In [ ]:
# Set the maximum number of iterations
max_iters = 6
# Initialize a counter
n = 0

# Create a DialogueSimulator object with the list of agents and the selection function
simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
# Reset the simulator to its initial state
simulator.reset()
# Inject the specified topic into the dialogue as a message from the moderator
simulator.inject("Moderator", specified_topic)
# Print the specified topic
print(f"(Moderator): {specified_topic}")
print("\n")

# Continue the dialogue until the maximum number of iterations is reached
while n < max_iters:
    # Get the name of the next speaker and their message
    name, message = simulator.step()
    # Print the name of the speaker and their message
    print(f"({name}): {message}")
    print("\n")
    # Increment the counter
    n += 1

(Moderator): To AI accelerationist: How can we ensure that automation and artificial intelligence advancements lead to increased employment opportunities rather than job displacement? Discuss potential strategies and policies to support job creation in the age of automation. 

To AI alarmist: What are the most effective ways to mitigate job displacement and address the challenges posed by automation and artificial intelligence on employment? Share your insights on potential solutions to support workers affected by technological advancements.




> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


```json
{
    "action": "duckduckgo_search",
    "action_input": "impact of automation and AI on employment"
}
```
Observation: The report also predicts two-thirds of jobs in the U.S. and Europe "are exposed to some degree of AI automation," and around a quarter of all jobs could be performed by AI entirely. Researchers from the University of Pennsylvania and OpenAI found some educated white-collar workers earning up to $80,000 a year are the most likely to be ... Based on panel data from 30 provinces in China from 2006 to 2020, a two-way fixed-effect model and the two-stage least squares method are used to analyse the impact of AI on employment and to ... However, we see generative AI enhancing the way STEM, creative, and business and legal professionals work rather than eliminating a significant number of jobs outright. Automation's biggest effects are likely to hit other job categories. Office support, customer service, and food service employment could continue to decline. The next

# The End